# Tensorflow学习之验证码识别

In [1]:
import tensorflow as tf

In [2]:
# 定义一个初始化权重的函数
def weght_variables(shape):
    w=tf.Variable(tf.random_normal(shape=shape,mean=0.0,stddev=1.0))
    return w

In [3]:
# 定义一个初始化偏置的函数
def bias_variables(shape):
    b=tf.Variable(tf.constant(0.0,shape=shape))
    return b

In [4]:
# 读取验证码图片
# 1、构建文件队列
file_queue=tf.train.string_input_producer(['Data\day_7\captcha.tfrecords'])

# 2、构建文件阅读器
reader=tf.TFRecordReader()

# 3、读取内容
key,value=reader.read(file_queue)

# tfrecords的格式example,需要解析
features=tf.parse_single_example(value,features={
    'image':tf.FixedLenFeature([],tf.string),
    'label':tf.FixedLenFeature([],tf.string)
})

# 解码内容，字符串
# 解码图片的特征值
image=tf.decode_raw(features['image'],tf.uint8)

# 解码图片的目标值
label=tf.decode_raw(features['label'],tf.uint8)

# 改变形状
image_reshape=tf.reshape(image,[20,80,3])
label_reshape=tf.reshape(label,[4])

# 进行批处理，每批次读取的样本数
image_batch,label_batch=tf.train.batch([image_reshape,label_reshape],batch_size=100,num_threads=1,capacity=100)

In [5]:
# 通过输入图片特征数据，建立模型，得出结果
# [100,20,80,3]*[20*80*3,4*26]+[104]=[100,4*26]
with tf.variable_scope('model'):
    # 1、随机初始化权重、偏置
    weights=weght_variables([20*80*3,4*26])
    bias=bias_variables([4*26])

    # 将数据形状转换为二维的
    image_reshape=tf.reshape(image,[100,20*80*3])

    # 进行全连接层运算
    y_predict=tf.matmul(tf.cast(image_reshape,tf.float32),weights)+bias

In [6]:
# 先把目标值转化为one_hot编码,提供给交叉熵损失计算
y_true=tf.one_hot(label_batch,depth=26,on_value=1.0,axis=2)

In [7]:
# 4、进行交叉熵损失计算
with tf.variable_scope('soft_cross'):
    # 求平均交叉熵损失,y_true [100,4,26]----->[100,4*26]
    loss=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf.reshape(y_true,[100,4*26]),logits=y_predict))

In [8]:
# 5、梯度下降求损失
with tf.variable_scope('optimizer'):
     train_op=tf.train.GradientDescentOptimizer(0.01).minimize(loss)

In [9]:
y_predict=tf.reshape(y_predict,[100,4,26])

In [10]:
# 6、进行准确率的计算,3维比较
with tf.variable_scope('acc'):
    # 比较预测值和目标值位置（4个）是否一样 y_predict[100,4*26]---->[100,4,26]
    equal_list=tf.equal(tf.argmax(y_true,2),tf.argmax(y_predict,2))

    accuracy=tf.reduce_mean(tf.cast(equal_list,tf.float32))

In [11]:
# 定义一个初始化op
init_op=tf.global_variables_initializer()

In [12]:
# 7、开启会话训练
with tf.Session() as sess:
    sess.run(init_op)

    # 定义线程协调器和开启线程
    coord=tf.train.Coordinator()

    # 开启线程去运作读取文件
    threads=tf.train.start_queue_runners(sess,coord=coord)

    # 训练识别程序
    for i in range(5000):
        sess.run(train_op)
        print("第%d批次的准确率为：%f" % (i, accuracy.eval()))

    # 回收线程
    coord.request_stop()
    coord.join(threads)

INFO:tensorflow:Error reported to Coordinator: <class 'tensorflow.python.framework.errors_impl.CancelledError'>, Enqueue operation was cancelled
	 [[Node: input_producer/input_producer_EnqueueMany = QueueEnqueueManyV2[Tcomponents=[DT_STRING], timeout_ms=-1, _device="/job:localhost/replica:0/task:0/cpu:0"](input_producer, input_producer/RandomShuffle)]]

Caused by op 'input_producer/input_producer_EnqueueMany', defined at:
  File "C:\User\Anaconda\envs\python35\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\User\Anaconda\envs\python35\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\User\Anaconda\envs\python35\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\User\Anaconda\envs\python35\lib\site-packages\traitlets\config\application.py", line 664, in launch_instance
    app.start()
  File "C:\User\Anaconda\envs\python35\lib\site-packages\ipykernel\kernelapp.py", line 61

InvalidArgumentError: Input to reshape is a tensor with 4800 values, but the requested shape has 480000
	 [[Node: model/Reshape = Reshape[T=DT_UINT8, Tshape=DT_INT32, _device="/job:localhost/replica:0/task:0/cpu:0"](DecodeRaw, model/Reshape/shape)]]

Caused by op 'model/Reshape', defined at:
  File "C:\User\Anaconda\envs\python35\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\User\Anaconda\envs\python35\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\User\Anaconda\envs\python35\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\User\Anaconda\envs\python35\lib\site-packages\traitlets\config\application.py", line 664, in launch_instance
    app.start()
  File "C:\User\Anaconda\envs\python35\lib\site-packages\ipykernel\kernelapp.py", line 612, in start
    self.io_loop.start()
  File "C:\User\Anaconda\envs\python35\lib\site-packages\tornado\platform\asyncio.py", line 149,